### Criar um secret scope

1º - Criar o secret no Key Vault da Azure

2º - Acessar o endereço "https://(databricks-instance-here)#secrets/createScope" colocando a sua instacia do databricks

3º - Preencher com as informações soicitadas
  
https://learn.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes  


### Como definir variaveis de ambiente no cluster

Na página de configuração do cluster, clique na alternância Opções Avançadas.

1º - Clique na guia Spark.

2º - Na configuração do Spark, insira as propriedades de configuração como um par chave-valor por linha, separados por espaço.

https://learn.microsoft.com/pt-br/azure/databricks/compute/configure#spark-configuration


### Acessar uma variavel de ambiente

In [0]:
sql_user = spark.conf.get("spark.sql_user")
sql_secret = spark.conf.get("spark.sql_secret")
server = spark.conf.get("server")
db = spark.conf.get("db")

### Montar um blob

```python
dbutils.fs.mount(
  source = "wasbs://<seu-container>@<seu-blob>.blob.core.windows.net",
  mount_point = "/mnt/blob/dax",
  extra_configs = {
    "fs.azure.account.key.<seu-blob>.blob.core.windows.net":
    dbutils.secrets.get(
      scope = "secret-scope", 
      key = "key"
      )
    }
  )
  ```

  https://learn.microsoft.com/en-us/azure/databricks/dbfs/mounts


### Bibliotecas importantes

In [0]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

### Leitura e transformação

In [0]:
df = spark.read.json("dbfs:/mnt/blob/dax*")

df_exp = df.select(
    'data.symbol',
    'data.ask',
    'data.baseVolume24h',
    'data.bid',
    'data.high24h',
    'data.lastPrice',
    'data.low24h',
    'data.open24h',
    'data.quoteVolume24h',
    'data.timestamp'
)

In [0]:
df_exp = (
    df_exp
    .withColumn("timestamp", from_unixtime(df_exp.timestamp / 1000))
    .withColumn("broker_id", lit("1"))
    .withColumn("usuario_id", lit("1"))
)

In [0]:
df = df_exp.select(
    'broker_id',
    'usuario_id',
    'symbol',
    'ask',
    'baseVolume24h',
    'bid',
    'high24h',
    'lastPrice',
    'low24h',
    'open24h',
    'quoteVolume24h',
    'timestamp'
)
# display(df)


### Filtrar o DF usando a data max da tabela que esta no SQL, recebida como parametro do pipeline

In [0]:
data = dbutils.widgets.get("max_date")
df = df.filter(col('timestamp') > data)

##Salvar no AzureSQL DB

In [0]:
jdbcUrl = 'jdbc:sqlserver://{0}:{1};database={2}'.format(server, 1433, db)
connectionProperties = {
  'user' : sql_user,
  'password' : sql_secret,
  'driver' : 'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}

table = 'dax_api'

(
  df.write.mode('append')
  .format('jdbc')
  .option('url', jdbcUrl)
  .option('dbtable', table)
  .option('user', sql_user)
  .option('password', sql_secret)
  .option('driver', jdbcDriver)
  .save()
)